# AORC Download and Storm Catalog

This notebook demonstrates how to download AORC (Analysis of Record for Calibration) precipitation data and build storm catalogs for HEC-HMS gridded precipitation modeling.

**Workflow Overview**:
1. Download HUC12 watersheds as study area templates (HmsHuc)
2. Generate storm catalog from AORC data (HmsAorc)
3. Download AORC precipitation data for selected storm
4. Convert to DSS grid format for HMS

**Prerequisites**:
- Internet connection (AWS S3 and USGS web services)
- Optional dependencies: `pip install hms-commander[all]`

**Series Navigation**:
- **14a** (this notebook): AORC download and storm catalog
- **14b**: HMS grid definition and HRAP mapping
- **14c**: HMS execution and results analysis

In [ ]:
# pip install hms-commander[all]

**For Development**: If working on hms-commander source code, use the `hmscmdr_local` conda environment (editable install) instead of pip install.

**Network Requirements**: This notebook requires internet access to:
- USGS WBD web services (HUC watershed boundaries)
- AWS S3 (AORC precipitation data - anonymous access)

## Setup and Imports

In [ ]:
from pathlib import Path
from datetime import datetime
import warnings
import logging

# Suppress warnings and verbose logging for cleaner output
warnings.filterwarnings('ignore')
logging.getLogger('botocore').setLevel(logging.WARNING)
logging.getLogger('botocore.httpchecksum').setLevel(logging.WARNING)
logging.getLogger('s3fs').setLevel(logging.WARNING)
logging.getLogger('aiobotocore').setLevel(logging.WARNING)

# HMS Commander imports
from hms_commander import HmsHuc, HmsAorc, HmsExamples

# Create output directory for this workflow
output_dir = Path("aorc_workflow_output")
output_dir.mkdir(exist_ok=True)

print(f"Output directory: {output_dir.absolute()}")
print(f"Workflow started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## Phase 1: Download HUC12 Watersheds

We'll use Bald Eagle Creek watershed in Pennsylvania as our study area. HmsHuc downloads HUC boundaries from USGS web services.

In [ ]:
# Study area bounds (west, south, east, north)
# Bald Eagle Creek, PA - a well-documented watershed
bounds = (-77.71, 41.01, -77.25, 41.22)

print("Study Area: Bald Eagle Creek, PA")
print(f"  West:  {bounds[0]}deg")
print(f"  South: {bounds[1]}deg")
print(f"  East:  {bounds[2]}deg")
print(f"  North: {bounds[3]}deg")
print(f"  Width:  {abs(bounds[2] - bounds[0]):.2f}deg (~{abs(bounds[2] - bounds[0]) * 85:.1f} km)")
print(f"  Height: {abs(bounds[3] - bounds[1]):.2f}deg (~{abs(bounds[3] - bounds[1]) * 111:.1f} km)")

In [ ]:
# Download HUC12 watersheds for study area
print("Downloading HUC12 watersheds from USGS WBD...")
watersheds = HmsHuc.get_huc12_for_bounds(bounds)

print(f"\nDownloaded {len(watersheds)} HUC12 watersheds")
print(f"Total area: {watersheds['areasqkm'].sum():.1f} km2")

In [ ]:
# Display watershed summary
display_cols = ['huc12', 'name', 'areasqkm']
watersheds[display_cols]

In [ ]:
# Get HUC information
huc_info = HmsHuc.get_huc_info()
print("HUC Level Information:")
for level, info in huc_info.items():
    print(f"  {level}: {info['description']} (typical size: {info['typical_size']})")

### Visualize Watersheds (Optional)

If geopandas and matplotlib are available, visualize the downloaded watersheds.

In [ ]:
try:
    import matplotlib.pyplot as plt
    
    fig, ax = plt.subplots(figsize=(12, 8))
    watersheds.plot(ax=ax, column='areasqkm', cmap='Blues', edgecolor='darkblue',
                    linewidth=1, alpha=0.7, legend=True,
                    legend_kwds={'label': 'Area (km2)'})
    
    # Label each watershed
    for idx, row in watersheds.iterrows():
        centroid = row.geometry.centroid
        ax.annotate(row['name'][:15], xy=(centroid.x, centroid.y),
                    ha='center', va='center', fontsize=7,
                    bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))
    
    ax.set_title('Bald Eagle Creek HUC12 Watersheds', fontsize=14, fontweight='bold')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    plt.tight_layout()
    plt.show()
    
except ImportError:
    print("Matplotlib not available - skipping visualization")

---

## Phase 2: AORC Storm Catalog

Generate a catalog of significant storm events from AORC data. This identifies storms with the highest precipitation depths for design storm analysis.

In [ ]:
# Get AORC dataset information
aorc_info = HmsAorc.get_info()
print("AORC Dataset Information:")
print(f"  Name: {aorc_info['name']}")
print(f"  Source: {aorc_info['source']}")
print(f"  Coverage: {aorc_info['coverage']['spatial']}")
print(f"  Resolution: {aorc_info['resolution']['spatial']}, {aorc_info['resolution']['temporal']}")
print(f"  Primary Variable: APCP_surface - {aorc_info['variables']['APCP_surface']}")

In [ ]:
# Generate storm catalog for 2019
# Using 2019 as a stable year with good data coverage
print("Generating storm catalog for 2019...")
print("(This may take 30-60 seconds to query AWS S3)")

storms = HmsAorc.get_storm_catalog(bounds, year=2019)

if len(storms) > 0:
    print(f"\nFound {len(storms)} significant storm events")
    print(f"Total precipitation depth range: {storms['total_depth_in'].min():.2f} - {storms['total_depth_in'].max():.2f} inches")
else:
    print("\nNo storms found - trying 2018 as fallback year...")
    storms = HmsAorc.get_storm_catalog(bounds, year=2018)
    if len(storms) > 0:
        print(f"Found {len(storms)} significant storm events in 2018")

In [ ]:
# Display top 10 storms by total depth
if len(storms) > 0:
    display_cols = ['storm_id', 'start_time', 'end_time', 'duration_hours', 'total_depth_in', 'rank']
    storms[display_cols].head(10)
else:
    print("No storm data available to display")

### Storm Catalog Visualization

Generate summary visualizations of all storm events.

In [ ]:
# Create storm catalog summary visualization
if len(storms) > 0:
    try:
        import matplotlib.pyplot as plt
        import numpy as np

        fig, axes = plt.subplots(2, 2, figsize=(14, 10))

        # 1. Total depth by storm
        ax1 = axes[0, 0]
        bars1 = ax1.bar(storms['storm_id'], storms['total_depth_in'], color='steelblue', alpha=0.8)
        ax1.set_xlabel('Storm ID')
        ax1.set_ylabel('Total Depth (inches)')
        ax1.set_title('Precipitation Totals by Storm')
        ax1.grid(True, alpha=0.3, axis='y')

        # 2. Peak intensity
        ax2 = axes[0, 1]
        bars2 = ax2.bar(storms['storm_id'], storms['peak_intensity_in_hr'], color='darkorange', alpha=0.8)
        ax2.set_xlabel('Storm ID')
        ax2.set_ylabel('Peak Intensity (in/hr)')
        ax2.set_title('Peak Rainfall Intensity')
        ax2.grid(True, alpha=0.3, axis='y')

        # 3. Duration
        ax3 = axes[1, 0]
        bars3 = ax3.bar(storms['storm_id'], storms['duration_hours'], color='seagreen', alpha=0.8)
        ax3.set_xlabel('Storm ID')
        ax3.set_ylabel('Duration (hours)')
        ax3.set_title('Storm Duration')
        ax3.grid(True, alpha=0.3, axis='y')

        # 4. Storm timeline
        ax4 = axes[1, 1]
        for i, (_, storm) in enumerate(storms.iterrows()):
            ax4.barh(i, storm['duration_hours'], left=storm['start_time'].dayofyear,
                     height=0.6, color='steelblue', alpha=0.7)
        ax4.set_yticks(range(len(storms)))
        ax4.set_yticklabels(storms['storm_id'])
        ax4.set_xlabel('Day of Year')
        ax4.set_title('Storm Timeline')
        ax4.grid(True, alpha=0.3, axis='x')

        plt.suptitle('Storm Catalog Summary', fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()

    except ImportError:
        print("Matplotlib not available - skipping visualization")
else:
    print("No storms available for visualization")

---

## Phase 3: Download AORC Data

Download AORC precipitation data for the selected (largest) storm event.

In [ ]:
# Select the largest storm for demonstration
if len(storms) > 0:
    largest_storm = storms[storms['rank'] == 1].iloc[0]

    print("Selected Storm (Largest):")
    print(f"  Storm ID: {largest_storm['storm_id']}")
    print(f"  Start: {largest_storm['start_time']}")
    print(f"  End: {largest_storm['end_time']}")
    print(f"  Duration: {largest_storm['duration_hours']} hours")
    print(f"  Total Depth: {largest_storm['total_depth_in']:.2f} inches")
    print(f"\nRecommended Simulation Window:")
    print(f"  Start: {largest_storm['sim_start']}")
    print(f"  End: {largest_storm['sim_end']}")
else:
    print("No storms available - using fixed date range for demonstration")
    # Fallback to a known good period
    largest_storm = {
        'storm_id': 'FALLBACK_2019',
        'sim_start': '2019-06-01',
        'sim_end': '2019-06-03',
        'total_depth_in': 2.0
    }

In [ ]:
# Download AORC data for the storm
nc_file = output_dir / "aorc_storm.nc"

print(f"Downloading AORC data from AWS S3...")
print(f"  Bounds: {bounds}")
if hasattr(largest_storm, 'keys'):
    print(f"  Period: {largest_storm['sim_start']} to {largest_storm['sim_end']}")
    sim_start = str(largest_storm['sim_start'])
    sim_end = str(largest_storm['sim_end'])
else:
    print(f"  Period: {largest_storm.sim_start} to {largest_storm.sim_end}")
    sim_start = str(largest_storm.sim_start)
    sim_end = str(largest_storm.sim_end)

nc_result = HmsAorc.download(
    bounds=bounds,
    start_time=sim_start,
    end_time=sim_end,
    output_path=nc_file
)

print(f"\nDownload complete!")
print(f"  File: {nc_result}")
print(f"  Size: {nc_result.stat().st_size / (1024*1024):.2f} MB")

In [ ]:
# Examine the NetCDF file structure
try:
    import xarray as xr
    
    ds = xr.open_dataset(nc_file)
    print("NetCDF Structure:")
    print(ds)
    
    # Get grid dimensions
    lon_dim = 'longitude' if 'longitude' in ds.dims else 'lon' if 'lon' in ds.dims else 'x'
    lat_dim = 'latitude' if 'latitude' in ds.dims else 'lat' if 'lat' in ds.dims else 'y'
    time_dim = 'time' if 'time' in ds.dims else 'valid_time'
    
    print(f"\nGrid Dimensions:")
    print(f"  Longitude: {len(ds[lon_dim])} cells ({float(ds[lon_dim].min()):.4f}deg to {float(ds[lon_dim].max()):.4f}deg)")
    print(f"  Latitude: {len(ds[lat_dim])} cells ({float(ds[lat_dim].min()):.4f}deg to {float(ds[lat_dim].max()):.4f}deg)")
    print(f"  Time: {len(ds[time_dim])} timesteps")
    
    ds.close()
except Exception as e:
    print(f"Could not examine NetCDF: {e}")

---

## Phase 4: Convert to DSS Grid

Convert the NetCDF to HMS-compatible DSS grid format using HEC Monolith libraries.

In [ ]:
# Convert NetCDF to DSS grid
dss_file = output_dir / "aorc_storm.dss"

print("Converting to DSS grid format...")
print("(Using HEC Monolith via pyjnius)")

dss_result = HmsAorc.convert_to_dss_grid(
    netcdf_file=nc_file,
    output_dss_file=dss_file,
    pathname="/AORC/BALDEAGLE/PRECIP////",
    units="MM"
)

print(f"\nConversion complete!")
print(f"  File: {dss_result}")
print(f"  Size: {dss_result.stat().st_size / (1024*1024):.2f} MB")

In [ ]:
# Verify DSS file (optional - requires ras-commander DSS support)
try:
    from ras_commander.dss import RasDss
    
    catalog = RasDss.get_catalog(str(dss_file))
    print(f"DSS Catalog: {len(catalog)} records")
    print(f"\nSample pathnames:")
    for path in catalog[:5]:
        print(f"  {path}")
    if len(catalog) > 5:
        print(f"  ... and {len(catalog) - 5} more")
except Exception as e:
    print(f"Could not read DSS catalog: {e}")
    print("(This is normal if ras-commander is not installed)")

---

## Summary

This notebook demonstrated:
1. Downloading HUC12 watershed boundaries using HmsHuc
2. Generating storm catalogs from AORC data using HmsAorc
3. Downloading AORC precipitation data for a selected storm
4. Converting to DSS grid format for HMS

**Output Files Created**:
- `aorc_storm.nc` - NetCDF precipitation data
- `aorc_storm.dss` - DSS grid for HMS

**Next Steps**:
- Continue to **14b_aorc_grid_setup.ipynb** to create HMS grid definition and cell mapping files

In [ ]:
# Summary of outputs
print("=" * 60)
print("PHASE 1-2 COMPLETE: AORC DOWNLOAD")
print("=" * 60)

print(f"\nStudy Area: Bald Eagle Creek, PA")
print(f"  Bounds: {bounds}")
print(f"  HUC12 watersheds: {len(watersheds)}")
print(f"  Total area: {watersheds['areasqkm'].sum():.1f} km2")

if len(storms) > 0:
    if hasattr(largest_storm, 'keys'):
        total_depth = largest_storm['total_depth_in']
    else:
        total_depth = largest_storm.total_depth_in
    print(f"\nSelected Storm:")
    print(f"  Total depth: {total_depth:.2f} inches")

print(f"\nOutput Files:")
for f in sorted(output_dir.glob("*")):
    if f.is_file():
        print(f"  {f.name}: {f.stat().st_size / 1024:.1f} KB")

print(f"\nWorkflow completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("\n--> Continue to 14b_aorc_grid_setup.ipynb")